In [1]:
import os
import pandas as pd
import numpy as np
import pandas_datareader
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
import yfinance as yf

def stockData(sList, stDate, endDate):
    """Take list of stocks and pull data for that stock and create technical indicator variables
    sList requires list of stocks, dates should be entered in format 'YYYY-MM-DD' """
   
    data = []
    for element in sList:
        try:
            temp = pdr.get_data_yahoo(element, start = stDate, end = endDate)
            #add an indentifier
            temp['Symbol'] = element
            temp['Close'] = temp['Close']
        except: 
            #If stock symbol cannot be found make it $1
            print (element)
            temp['Symbol'] = element
            temp['Close'] = 1
        else:
        #add data to list 
            data.append(temp)
        #convert to dataframe    
        df = pd.concat(data)
        #drop rows for which we will not have all data points calculated
        df = df.dropna()
        #dates above what user entered
        df = df.loc[stDate : endDate]
        #order and limit columns to those needed
        df = df.loc[:, ['Symbol', 'Close']]
        df['Day'] = pd.to_datetime(df.index)

    
    return(df)


In [2]:
new_base = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/full_combine.csv")

In [3]:
#List of all ticker symbols
symbols = new_base['Symbol'].drop_duplicates()
symbols.head(10)

0     AAPL
1    AGTHX
2     AMZN
3      CHE
4     DOCU
5    GSGRX
6     ICLN
7    OEGAX
8     PBFS
9      QQQ
Name: Symbol, dtype: object

# Simple Comparison (Initial vs Current Value) START HERE TO UPDATE

In [4]:
#Select date to compare to initial prices
cur_prices = stockData(list(symbols), '2019-10-11', '2019-10-11')

In [5]:
cur_prices.sort_values("Symbol", inplace = True)
cur_prices.drop_duplicates(subset = "Symbol", inplace = True)

In [6]:
cur_prices.to_csv("cur_prices.csv")

In [25]:
#cur_prices= pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/cur_prices.csv")
#cur_prices.head()


,Date,Symbol,Close,Day
0,2019-09-27,AAL,27.010000,2019-09-27
1,2019-09-27,AAPL,218.820007,2019-09-27
2,2019-09-27,ACB,4.620000,2019-09-27
3,2019-09-27,ACER,3.300000,2019-09-27
4,2019-09-27,ACST,1.960000,2019-09-27


In [7]:
detail = pd.merge(new_base, cur_prices, how = 'left', on = "Symbol", validate="many_to_one")

In [8]:
detail.head()

,Unnamed: 0,Port_Num,New Cost Basis,Name,Symbol,% of Portfolio,Cost Basis,Close_x,Day_x,# of Shares,initial_basis,Close_y,Day_y
0,0,1,194413.7192,Adrienne Villano,AAPL,0.07,14000.0,213.259995,2019-09-06 00:00:00,65.647568,200000,236.210007,2019-10-11
1,1,1,194413.7192,Adrienne Villano,AGTHX,0.11,22000.0,50.000000,2019-09-06 00:00:00,440.000000,200000,49.590000,2019-10-11
2,2,1,194413.7192,Adrienne Villano,AMZN,0.07,14000.0,1833.510010,2019-09-06 00:00:00,7.635628,200000,1731.920044,2019-10-11
3,3,1,194413.7192,Adrienne Villano,CHE,0.18,36000.0,439.820007,2019-09-06 00:00:00,81.851665,200000,417.119995,2019-10-11
4,4,1,194413.7192,Adrienne Villano,DOCU,0.05,10000.0,56.270000,2019-09-06 00:00:00,177.714589,200000,66.529999,2019-10-11


In [9]:
detail['Current_Value'] = detail['Close_y'] * detail['# of Shares']
detail['Perc_Return'] = detail['Close_y']/detail['Close_x']  -1
detail.head(5)

,Unnamed: 0,Port_Num,New Cost Basis,Name,Symbol,% of Portfolio,Cost Basis,Close_x,Day_x,# of Shares,initial_basis,Close_y,Day_y,Current_Value,Perc_Return
0,0,1,194413.7192,Adrienne Villano,AAPL,0.07,14000.0,213.259995,2019-09-06 00:00:00,65.647568,200000,236.210007,2019-10-11,15506.612488,0.107615
1,1,1,194413.7192,Adrienne Villano,AGTHX,0.11,22000.0,50.000000,2019-09-06 00:00:00,440.000000,200000,49.590000,2019-10-11,21819.600067,-0.008200
2,2,1,194413.7192,Adrienne Villano,AMZN,0.07,14000.0,1833.510010,2019-09-06 00:00:00,7.635628,200000,1731.920044,2019-10-11,13224.296833,-0.055407
3,3,1,194413.7192,Adrienne Villano,CHE,0.18,36000.0,439.820007,2019-09-06 00:00:00,81.851665,200000,417.119995,2019-10-11,34141.966200,-0.051612
4,4,1,194413.7192,Adrienne Villano,DOCU,0.05,10000.0,56.270000,2019-09-06 00:00:00,177.714589,200000,66.529999,2019-10-11,11823.351384,0.182335


In [10]:
detail.to_csv("pt2-detail.csv")

### Individual Performance

In [11]:
ranking = detail.groupby("Name")['Current_Value'].sum().reset_index().sort_values("Current_Value", ascending=False)

In [12]:
ranking['Perc_Return'] = ranking['Current_Value']/200000 -1
ranking['Rank'] = ranking['Perc_Return'].rank(ascending=False)
ranking.head(50)

,Name,Current_Value,Perc_Return,Rank
12,Colton Young,215259.639412,0.076298,1.0
25,Joelean Mcbride,214262.731528,0.071314,2.0
26,Joseph Lindner,210405.148244,0.052026,3.0
28,Kelly Nicholson,208488.586651,0.042443,4.0
9,Bradley Bonfante,206040.045801,0.030200,5.0
36,Marie Barkamian,204188.219947,0.020941,6.0
20,Estelle Genest,203107.614891,0.015538,7.0
5,Anil Mulchandani,202793.657093,0.013968,8.0
19,Elise Shaver,202535.773872,0.012679,9.0
49,Sharon Strock,201858.745468,0.009294,10.0


In [108]:
ranking.to_csv("individual rankings.csv")

### Best Individual Stock Returns

In [109]:
best_stocks = detail.drop_duplicates(['Symbol']).reset_index().sort_values(['Perc_Return'], ascending=False)

In [110]:
best_stocks = best_stocks.loc[:, ['Symbol', 'Perc_Return']]
best_stocks.head(10)

,Symbol,Perc_Return
63,PIR,0.514610
111,HOME,0.277624
167,DRIP,0.172521
96,VHC,0.170094
153,RVNC,0.142594
113,RH,0.137050
171,WORK,0.130100
4,DOCU,0.105562
28,BEP,0.104757
183,GLUU,0.087755


In [111]:
best_stocks.to_csv("stock ranking.csv")

# Trend

In [ ]:
#Pull Only necessary columns
combined = combined.loc[:, ['Name', 'Symbol', '# of Shares']]

In [ ]:
Price_History = stockData(stocks['Ticker'], '2019-08-01', '2019-09-06')

In [ ]:
historical = pd.merge(combined, Price_History, how = 'outer', on = "Symbol", validate="many_to_many")


In [ ]:
historical['Value'] = historical['# of Shares'] * historical['Close']
historical.head()

In [ ]:
trend_data = historical.groupby(['Name', 'Day'])['Value'].sum().reset_index().sort_values(['Name', 'Day'])

In [ ]:
trend_data.head()

In [1]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode